In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from sklearn.decomposition import IncrementalPCA, MiniBatchNMF
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, make_scorer

import random
import pickle
import joblib

import data_splitter as ds
import data_loader as dl
import feature_extraction as fex
import run_model as rm

In [3]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV

## Get 25% sample

In [ ]:
splitter = ds.DataSplitter(verbose=True)

In [6]:
# image id of all sample training set after split to calib and train set

train_img_ids = [id for k, v in splitter.trainset.items() for id in v]
print(len(train_img_ids))

35003


In [7]:
metadata = pd.read_csv('train.csv')

In [8]:
# stratified sampling 25% from the training set >> 8,751 images
sample = metadata[metadata['patient_id'].isin(splitter.trainset.keys())]
sample = sample.groupby('cancer', group_keys=False).apply(lambda x: x.sample(frac=0.25))
sample_img_ids = list(sample.image_id.unique())

print(len(sample.patient_id.unique()), len(sample_img_ids))

5507 8751


# Hyperparameter search

Using NMF + logistic regression pipeline, optimizing for average precision
Only 256 x 256 resolution is used because scikit-learn search algorithms do not support batch learning and higher resolution requires exceedingly high computational memory requirement.

## Image Resolution 256 x 256

In [13]:
BASEPATH = '/scratch/siads699w23_class_root/siads699w23_class/shared_data/team_mammogram/train_images'

# scitkit-learn parameter search does not support batch learning
# so we'll have to combine data in 1 batch
BATCH_SIZE = 8751 

# from PCA, image resolution does not seem to have any effect on no. of component
# so we'll use the lowest resolution to minimize training cost
IMG_SIZE = (256, 256, 1)

RANDOM_STATE = 42

# define DataGenerators for training split
train_gen = dl.DataGenerator(
    list_IDs = sample_img_ids,
    labels = splitter.labels,
    patient_img_dict = splitter.trainset,
    basepath = BASEPATH,
    batch_size = BATCH_SIZE,
    img_size = IMG_SIZE,
    n_classes = 2,
    shuffle = True,
    normalize = (0, 1),
    feature_extractor = None,
    verbose = True
    )

# check no. of batch
print(len(train_gen))

1


In [10]:
# get X and y
for X, y in train_gen:
    X = X.reshape(BATCH_SIZE, -1)
    y = y[:, 1]
    print(X.shape, y.shape)

(8751, 65536) (8751,)


In [11]:
# with open('sample_flatten_X.npy','wb') as f:
#     np.save(f, X)
    
# with open('sample_flatten_y.npy','wb') as f:
#     np.save(f, y)

In [4]:
X = np.load('sample_flatten_X.npy')
y = np.load('sample_flatten_y.npy')

In [5]:
from collections import Counter
Counter(y)

Counter({0.0: 8570, 1.0: 181})

In [10]:
nmf = MiniBatchNMF(
    max_iter = 10000,
    tol = 1e-4, #control early stopping based on changes in H
    max_no_improvement = 3, #control early stopping based on smoothed cost function
    forget_factor = 1, #rescaling factor of past
    fresh_restarts = False, #completely solve for W at each step 
    random_state = 42
    )

logit = LogisticRegression(
    warm_start = True,
    solver = 'saga',
    max_iter = 10000,
    l1_ratio = 0.5,
    random_state = 42
    )

pipeline = Pipeline([
    ('feature_extraction', nmf),
    ('classify', logit)
])

param_grid = {
    'feature_extraction__n_components': np.arange(2, 51, step=1),
    'feature_extraction__init': ['random', 'nndsvd', 'nndsvda'],
    'feature_extraction__beta_loss': ['frobenius', 'kullback-leibler'],
    'feature_extraction__alpha_W': np.arange(0, 1, step=0.1), #np.logspace(0, -4, num=5, base=10, endpoint=True),
    'feature_extraction__l1_ratio': np.arange(0, 1, step=0.1)
}

In [12]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=42, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/l

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/l

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=10; total time= 7.4min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=18; total time=23.5min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=19; total time=  13.2s
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=2; total time= 6.5min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.0, featu

...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=36.3min
[CV] END feature_extraction__alpha_W=0.5, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=45; total time=37.5min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=38.8min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=39.2min


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=41.7min


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=10000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=10000,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),...
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [13]:
print(grid.best_params_)

{'feature_extraction__n_components': 45, 'feature_extraction__l1_ratio': 0.30000000000000004, 'feature_extraction__init': 'nndsvd', 'feature_extraction__beta_loss': 'kullback-leibler', 'feature_extraction__alpha_W': 0.5}


In [14]:
print(grid.best_score_)

0.04368232656274275


In [15]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400__round8_randomstate42.pkl')

['models/nmf_hp_search_256_minre400__round8_randomstate42.pkl']

[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=47.4min


In [18]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=18, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/l

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-pa

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3

[CV] END feature_extraction__alpha_W=0.5, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=19; total time= 6.5min
[CV] END feature_extraction__alpha_W=0.5, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=45; total time=20.2min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.5, feature_extraction__n_components=36; total time= 4.1min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=19; total time=  11.0s
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, fe

...

[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=10; total time=12.2min


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=16; total time=15.2min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=52.7min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=10; total time=13.4min
[CV] END feature_extraction__alpha_W=0.5, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=19; total time= 4.6min
[CV] END feature_extraction__alpha_W=0.2, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=18, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=10000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=10000,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),...
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [19]:
print(grid.best_params_)

{'feature_extraction__n_components': 2, 'feature_extraction__l1_ratio': 0.0, 'feature_extraction__init': 'nndsvda', 'feature_extraction__beta_loss': 'kullback-leibler', 'feature_extraction__alpha_W': 0.1}


In [20]:
print(grid.best_score_)

0.027672919908400363


In [21]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400__round10_randomstate18.pkl')

['models/nmf_hp_search_256_minre400__round10_randomstate18.pkl']

[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=19; total time=37.0min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=19; total time=37.4min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=19; total time=40.0min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=19; total time=40.3min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__

In [ ]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=55, min_resources=400, verbose=2)

In [ ]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=300, min_resources=400, verbose=2)

In [22]:
print(grid.best_params_)

{'feature_extraction__n_components': 46, 'feature_extraction__l1_ratio': 0.1, 'feature_extraction__init': 'nndsvda', 'feature_extraction__beta_loss': 'kullback-leibler', 'feature_extraction__alpha_W': 0.30000000000000004}


In [23]:
print(grid.best_score_)

0.05433859842079419


In [24]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400__round7_randomstate300.pkl')

['models/nmf_hp_search_256_minre400__round7_randomstate300.pkl']

[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=16; total time=16.8min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=51.4min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=46; total time=60.2min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=46; total time=61.0min


In [16]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=699, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/met

...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time= 8.5min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time= 8.4min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time= 8.7min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time= 8.9min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=50.1min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=50.2min


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=699, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      f...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [17]:
print(grid.best_params_)

{'feature_extraction__n_components': 27, 'feature_extraction__l1_ratio': 0.5, 'feature_extraction__init': 'random', 'feature_extraction__beta_loss': 'frobenius', 'feature_extraction__alpha_W': 0.30000000000000004}


In [18]:
print(grid.best_score_)

0.032522053773854


In [19]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400__round7_randomstate699.pkl')

['models/nmf_hp_search_256_minre400__round7_randomstate699.pkl']

[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=51.0min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=21; total time=15.0min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=17; total time= 4.5min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=28; total time= 8.7min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extr

In [12]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=101, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

...

[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=50.1min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=18; total time=25.4min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=13; total time= 3.4min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=24.0min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=n

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=3; total time=20.8min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=6; total time= 7.7min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=27.2min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.8, feature_extraction__n_components=26; total time=67.3min


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=101, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      f...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [13]:
print(grid.best_params_)

{'feature_extraction__n_components': 23, 'feature_extraction__l1_ratio': 0.4, 'feature_extraction__init': 'random', 'feature_extraction__beta_loss': 'kullback-leibler', 'feature_extraction__alpha_W': 0.1}


In [14]:
print(grid.best_score_)

0.04655414841168275


In [15]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400_round6_randomstate101.pkl')

['models/nmf_hp_search_256_minre400__factor3_round6.pkl']

[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.5, feature_extraction__n_components=27; total time= 7.8min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=12.3min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=28; total time= 8.6min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=23; total time=23.0min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init

In [10]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=321, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.8, feature_extraction__n_components=26; total time=64.0min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.8, feature_extraction__n_components=26; total time=67.9min


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.8, feature_extraction__n_components=26; total time=74.9min


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=321, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      f...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [11]:
print(grid.best_params_)

{'feature_extraction__n_components': 21, 'feature_extraction__l1_ratio': 0.4, 'feature_extraction__init': 'random', 'feature_extraction__beta_loss': 'frobenius', 'feature_extraction__alpha_W': 0.9}


In [12]:
print(grid.best_score_)

0.05020124042943165


In [13]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400_round5_randomstate321.pkl')

['models/nmf_hp_search_256_minre400_round5_randomstate321.pkl']

In [7]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=123, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

...

[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=30.6min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=31.4min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=31.6min


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=35.6min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=36.5min


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=123, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      f...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [8]:
print(grid.best_params_)

{'feature_extraction__n_components': 30, 'feature_extraction__l1_ratio': 0.6000000000000001, 'feature_extraction__init': 'nndsvd', 'feature_extraction__beta_loss': 'kullback-leibler', 'feature_extraction__alpha_W': 0.4}


In [9]:
print(grid.best_score_)

0.024475719261660978


In [10]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400_round4_randomstate123.pkl')

['models/nmf_hp_search_256_minre400_round4_randomstate123.pkl']

In [7]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=0, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/lin

...

[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=17; total time= 8.7min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=17; total time= 9.0min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time= 9.1min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=17; total time= 9.7min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_ex

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=30; total time=13.6min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=30; total time=14.4min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=30; total time=15.4min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=30; total time=19.0min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      fac...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [8]:
print(grid.best_params_)

{'feature_extraction__n_components': 17, 'feature_extraction__l1_ratio': 0.6000000000000001, 'feature_extraction__init': 'nndsvda', 'feature_extraction__beta_loss': 'frobenius', 'feature_extraction__alpha_W': 0.0}


In [9]:
print(grid.best_score_)

0.04463876284831676


In [10]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400_round3_randomstate0.pkl')

['models/nmf_hp_search_256_minre400_round3_randomstate0.pkl']

[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=18; total time=43.4min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=18; total time=43.8min


In [9]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=42, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

...

[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time= 8.5min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=21; total time=25.6min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=28; total time= 7.1min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=13; total time= 2.7min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extr

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.5, feature_extraction__n_components=27; total time=14.9min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.5, feature_extraction__n_components=27; total time=14.8min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.5, feature_extraction__n_components=27; total time=15.2min


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      fa...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=3; total time=15.8min


In [10]:
print(grid.best_params_)

{'feature_extraction__n_components': 27, 'feature_extraction__l1_ratio': 0.5, 'feature_extraction__init': 'random', 'feature_extraction__beta_loss': 'frobenius', 'feature_extraction__alpha_W': 0.30000000000000004}


In [11]:
print(grid.best_score_)

0.037227509575818905
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=3; total time=20.4min


In [12]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400.pkl')

['models/nmf_hp_search_256_minre400.pkl']

In [7]:
grid = rm.hp_search(X, y, pipeline, param_grid, random_state=42, min_resources=400, verbose=2)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

...

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.5, feature_extraction__n_components=27; total time= 8.7min
[CV] END feature_extraction__alpha_W=0.8, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.7000000000000001, feature_extraction__n_components=12; total time= 3.6min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=15.8min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=28; total time=10.2min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extracti

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=3; total time=16.3min


HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      fa...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [8]:
print(grid.best_params_)

{'feature_extraction__n_components': 27, 'feature_extraction__l1_ratio': 0.5, 'feature_extraction__init': 'random', 'feature_extraction__beta_loss': 'frobenius', 'feature_extraction__alpha_W': 0.30000000000000004}


In [9]:
print(grid.best_score_)

0.037227509575818905


In [10]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400_round2.pkl')

['models/nmf_hp_search_256_minre400_round2.pkl']

[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=3; total time=21.8min


In [12]:
RANDOM_STATE = 42

grid = HalvingRandomSearchCV(
    pipe,
    cv = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    param_distributions = param_grid,
    
    factor = 5, # 1/5 candidates selected in each iteration
    aggressive_elimination = True,
    n_candidates = 'exhaust',
    scoring = make_scorer(average_precision_score, needs_proba=True),
    refit = False,
    
    n_jobs = -1,
    random_state = 42,
    verbose = 2
)
    
grid.fit(X, y)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 20
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 437
n_resources: 20
Fitting 5 folds for each of 437 candidates, totalling 2185 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packag

[CV] END feature_extraction__alpha_W=0.8, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=30; total time=   0.0s
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=6; total time=  24.9s
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=24; total time=   0.0s
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=24; total time=   0.0s
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_r

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packag

[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=19; total time= 1.5min
[CV] END feature_extraction__alpha_W=0.7000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=20; total time=   0.0s
[CV] END feature_extraction__alpha_W=0.7000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=20; total time=   0.0s
[CV] END feature_extraction__alpha_W=0.7000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=20; total time=   0.0s
[CV] END feature_extraction__alpha_W=0.7000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__ini

...

[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.7000000000000001, feature_extraction__n_components=16; total time= 3.9min
[CV] END feature_extraction__alpha_W=0.8, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.8, feature_extraction__n_components=7; total time= 5.7min
[CV] END feature_extraction__alpha_W=0.7000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.8, feature_extraction__n_components=8; total time= 6.9min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=14; total time= 2.2min
[CV] END feature_extraction__alpha_W=0.4, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, 

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.8, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=3; total time= 2.7min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=13; total time= 2.8min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=16; total time= 7.7min
[CV] END feature_extraction__alpha_W=0.2, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.4, feature_extraction__n_components=17; total time=14.5min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nnds

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
      

HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=50000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      fac...
                                           'feature_extraction__init': ['random',
                                                                        'nndsvd',
                                                                        'nndsvda'],
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [13]:
print(grid.best_params_)

{'feature_extraction__n_components': 16, 'feature_extraction__l1_ratio': 0.0, 'feature_extraction__init': 'random', 'feature_extraction__beta_loss': 'kullback-leibler', 'feature_extraction__alpha_W': 0.9}


In [14]:
print(grid.best_score_)

0.03679853869362119


In [15]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256.pkl')

['models/nmf_hp_search_256.pkl']